In [1]:
from ThesisPackage.RL.agent import Agent
from ThesisPackage.Environments.multi_pong_language import PongEnv
import torch
import numpy as np
import time
import os
import seaborn as sns
import matplotlib.pyplot as plt
import random

seed = 1
num_envs = 1

sequence_length = 2
vocab_size = 3

def replace_with_random_list(original_list, vocab_size):
    """
    Replaces the contents of the original list with a random list of integers of the same size.
    
    Parameters:
    - original_list: The list to be replaced.
    
    Returns:
    A new list with random integers of the same length as the original list.
    """
    # Get the length of the original list
    length = len(original_list)
    
    # Generate a new list of random integers
    # For simplicity, the range of integers is set from 0 to 100, but you can adjust this range as needed
    random_list = [random.randint(0, vocab_size) for _ in range(length)]
    
    return random_list

env = PongEnv(width=20, height=20, sequence_length=sequence_length, vocab_size=vocab_size, max_episode_steps=2048)
env.action_space.seed(seed)
env.observation_space.seed(seed)

agent = Agent(env)
agent.load_state_dict(torch.load('models/Pong__Language experiment__1__1706907676.pt'))
agent.eval()

/home/cowolff/miniconda3/envs/language/lib/python3.10/site-packages/gym/spaces/box.py:84: UserWarning: WARN: Box bound precision lowered by casting to float16
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


Agent(
  (critic): Sequential(
    (0): Linear(in_features=12, out_features=128, bias=True)
    (1): Tanh()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): Tanh()
    (4): Linear(in_features=64, out_features=1, bias=True)
  )
  (actor): Sequential(
    (0): Linear(in_features=12, out_features=128, bias=True)
    (1): Tanh()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): Tanh()
    (4): Linear(in_features=64, out_features=9, bias=True)
  )
)

In [2]:
def test_replace(replace=True, threshold=2, bigger=False, epochs=100):
    lengths = []
    distances = []
    tokens = {i: [] for i in range(sequence_length)}
    total_timesteps = 0
    replaced_timesteps = 0
    for i in range(epochs):
        obs = env.reset()
        obs = np.array([obs[agent] for agent in env.agents])
        obs = torch.Tensor(obs)
        done = [False]
        reached_distance = False
        while not any(done):
            action, logprob, _, value = agent.get_action_and_value(obs)
            action = action.cpu().numpy()

            total_timesteps += 1

            distance = abs(env.paddles["paddle_1"] - env.paddles["paddle_2"])

            utterances = action[0][1:]
            
            if bigger and replace:
                if distance < threshold:
                    utterances = replace_with_random_list(utterances, vocab_size)
                    action[0][1:] = utterances
                    replaced_timesteps += 1
            elif replace:
                if distance > threshold:
                    utterances = replace_with_random_list(utterances, vocab_size)
                    action[0][1:] = utterances
                    replaced_timesteps += 1

            if distance > threshold:
                reached_distance = True

            for i, utterance in enumerate(utterances):
                tokens[i].append(utterance)

            action = {agent: action[i] for i, agent in enumerate(env.agents)}
            
            distance = abs(env.paddles["paddle_1"] - env.paddles["paddle_2"])
            distances.append(distance)

            obs, rewards, terminated, truncated, info = env.step(action)
            obs = np.array([obs[agent] for agent in env.agents])
            obs = torch.Tensor(obs)
            done = {key: terminated.get(key, False) or truncated.get(key, False) for key in set(terminated) | set(truncated)}
            done = [done[agent] for agent in env.agents]
            if any(done):
                if reached_distance:
                    lengths.append(env.timestep)
                break
    
    print("\n")
    print("Mean episode length:", np.mean(lengths))
    print("total timesteps:", total_timesteps, "replaced timesteps:", replaced_timesteps, "percentage replaced:", replaced_timesteps / total_timesteps)

In [3]:
test_replace(replace=True, threshold=2, bigger=False, epochs=200)
test_replace(replace=True, threshold=2, bigger=True, epochs=200)
test_replace(replace=False, threshold=2, bigger=False, epochs=200)



Mean episode length: 572.21
total timesteps: 114442 replaced timesteps: 32951 percentage replaced: 0.28792750913126297


Mean episode length: 647.04
total timesteps: 129408 replaced timesteps: 70451 percentage replaced: 0.544409928288823


Mean episode length: 701.805
total timesteps: 140361 replaced timesteps: 0 percentage replaced: 0.0
